# Data Validation

## Preparation data

Read the CSV file and create a whole data frame. Since the initial data type is 'str', correct some needed fields, which not only contains 'int', but also includes 'bit', 'tinyint', and 'decimal' to 'int'.

In [1016]:
!pip install pandera numpy pandas matplotlib
import pandera as pa
from pandera import Column, Check, DataFrameSchema
import numpy as np
import pandas as pd

DTYPE = {
    'Crash ID': int,
    'Record Type': int,
    'City Section ID': int,
    'Urban Area Code': int,
    'NHS Flag': int,
    'Latitude Degrees': int,
    'Latitude Minutes': int,
    'Latitude Seconds': int,
    'Longitude Degrees': int,
    'Longitude Minutes': int,
    'Longitude Seconds': int,
}

# source data file path
DATA_PATH = "./data.csv"

# import as dataframe
dfori = pd.read_csv(DATA_PATH)

# change some normal column data type
df = dfori.astype(DTYPE, copy=True, errors='ignore')

df.head()

,Crash ID,Record Type,Vehicle ID,Participant ID,Participant Display Seq#,Vehicle Coded Seq#,Participant Vehicle Seq#,Serial #,Crash Month,Crash Day,Crash Year,Week Day Code,Crash Hour,County Code,City Section ID,Urban Area Code,Functional Class Code,NHS Flag,Highway Number,Highway Suffix,Roadway Number,Highway Component,Mileage Type,Connection Number,Linear Reference System (LRS),Latitude Degrees,Latitude Minutes,Latitude Seconds,Longitude Degrees,Longitude Minutes,Longitude Seconds,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Special Jurisdiction,Jurisdiction Group,Street Number,Nearest Intersecting Street Number,Intersection Sequence Number,Distance from Intersection,Direction From Intersection,...,Vehicle Cause 3 Code,Vehicle Event 1 Code,Vehicle Event 2 Code,Vehicle Event 3 Code,Vehicle Exceeded Posted Speed Flag,Vehicle Hit & Run Flag,Safety Equipment Used Quantity,Safety Equipment Un-used Quantity,Safety Equipment Use Unknown Quantity,Vehicle Occupant Count,Vehicle Striking Flag,Participant Type Code,Participant Hit & Run Flag,Public Employee Flag,Sex,Age,Driver License Status,Driver Residence Status,Injury Severity,Participant Safety Equipment Use Code,Airbag Deployment,Non-Motorist Movement Code,Non-Motorist Travel Direction From,Non-Motorist Travel Direction To,Non-Motorist Location,Participant Action,Participant Error 1 Code,Participant Error 2 Code,Participant Error 3 Code,Participant Cause 1 Code,Participant Cause 2 Code,Participant Cause 3 Code,Participant Event 1 Code,Participant Event 2 Code,Participant Event 3 Code,BAC Test Results Code,Alcohol Use Reported,Drug Use Reported,Participant Marijuana Use Reported,Participant Striker Flag
0,1809119,1,NaN,NaN,NaN,NaN,NaN,99992.0,3.0,14.0,2019.0,5.0,14.0,26.0,0.0,57.0,14.0,1.0,26.0,NaN,1.0,0.0,0.0,NaN,002600100S00,45.0,27.0,40.62,-122.0,22.0,59.23,0.0,NaN,NaN,NaN,NaN,NaN,9.0,16.77,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1809119,2,3409578.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1809119,3,3409578.0,3887884.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,20.0,1.0,1.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,47.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,9.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1809119,2,3409579.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,1.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1809119,3,3409579.0,3887885.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,66.0,1.0,1.0,9.0,2.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,0.0,NaN,NaN,NaN,NaN,NaN


## Validation

In week 5, I added this part and change one table to three proper tables.

In [1017]:
# devide one table to three tables.
# for crash record
crash = df[df['Record Type'] == 1]
# for vehicles record
vehicle = df[df['Record Type'] == 2]
# for participants record
participant = df[df['Record Type'] == 3]


# vertify split correctly.
assert crash.index.size + vehicle.index.size + participant.index.size == df.index.size

crash = crash.dropna(axis=1,how='all')
vehicle = vehicle.dropna(axis=1,how='all')
participant = participant.dropna(axis=1,how='all')


participant.head()

,Crash ID,Record Type,Vehicle ID,Participant ID,Participant Display Seq#,Vehicle Coded Seq#,Participant Vehicle Seq#,Safety Equipment Un-used Quantity,Safety Equipment Use Unknown Quantity,Vehicle Occupant Count,Vehicle Striking Flag,Participant Type Code,Participant Hit & Run Flag,Public Employee Flag,Sex,Age,Driver License Status,Driver Residence Status,Injury Severity,Participant Safety Equipment Use Code,Airbag Deployment,Non-Motorist Movement Code,Non-Motorist Travel Direction From,Non-Motorist Travel Direction To,Non-Motorist Location,Participant Action,Participant Error 1 Code,Participant Error 2 Code,Participant Error 3 Code,Participant Cause 3 Code,Participant Event 1 Code,Participant Event 2 Code,Participant Event 3 Code
2,1809119,3,3409578.0,3887884.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,20.0,1.0,1.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,47.0,NaN,NaN,1.0,NaN,NaN,NaN,0.0,9.0,0.0,0.0
4,1809119,3,3409579.0,3887885.0,2.0,2.0,1.0,1.0,0.0,0.0,1.0,66.0,1.0,1.0,9.0,2.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,9.0,9.0,0.0
7,1809229,3,3409765.0,3888073.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,41.0,2.0,4.0,9.0,2.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
8,1809229,3,0.0,3888074.0,2.0,NaN,1.0,3.0,0.0,0.0,2.0,32.0,NaN,NaN,1.0,NaN,NaN,1.0,0.0,0.0,4.0,50.0,70.0,NaN,NaN,18.0,NaN,NaN,NaN,0.0,9.0,1.0,0.0
11,1809637,3,3410470.0,3888766.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,90.0,2.0,4.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,0.0,47.0,81.0,NaN,1.0,NaN,NaN,NaN,0.0,9.0,0.0,0.0


Begin valiadtion.

### Existence

In [1018]:
# Every record has a record ID
assert df['Crash ID'].notna().all() == True

# Every record has a record type
assert df['Record Type'].notna().all() == True

# Every participant has an age
age_exist_vali = DataFrameSchema({
    'Age': Column(pa.String, nullable=False),
})
try:
  age_exist_vali.validate(participant)
except pa.errors.SchemaError as err:
  age_exist_err = err.failure_cases

age_exist_err

,index,failure_case
0,8,NaN
1,15,NaN
2,60,NaN
3,61,NaN
4,62,NaN
5,92,NaN
6,272,NaN
7,667,NaN
8,668,NaN
9,818,NaN


In [1019]:
# resolve above problem:
type(age_exist_err.index)

pandas.core.indexes.range.RangeIndex

### Limit

In [1020]:
# The record type should be 1, 2, or 3.
def check_record_type(x):
  n = x['Record Type']
  if (n == 1) or (n == 2) or (n == 3):
    pass
  else:
    return x


df.apply(check_record_type, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
2734    None
2735    None
2736    None
2737    None
2738    None
Length: 2739, dtype: object

In [1021]:
# The day, month, and year should be a valid range, from 01/01/2019 to 12/31/2019.
crash_date = DataFrameSchema({
    'Crash Year': Column(pa.String, Check(lambda n: n == '2019')),
    'Crash Month': Column(pa.String, Check.isin([str(x) for x in range(1, 13)])),
    'Crash Day': Column(pa.String, Check.isin([str(x) for x in range(1, 32)])),
})
try:
  crash_date.validate(crash)
except pa.errors.SchemaError as err:
  print(err.failure_cases)

  index failure_case
0  None      float64


In [1022]:
# The crash hours should be valid char, from 00 to 23 or 99.
crash_hour_vali = DataFrameSchema({
    'Crash Hour': Column(pa.String, Check.isin(['%.2d' % i for i in range(24)] + ['99']))
})
try:
  crash_hour_vali.validate(crash)
except pa.errors.SchemaError as err:
  crash_hour_err = err.failure_cases

crash_hour_err

,index,failure_case
0,None,float64


In [1023]:
# The crash day of week should be in a valid range, from 1 to 7.
crash_day_vali = DataFrameSchema({
    'Week Day Code': Column(pa.String, Check.isin([str(x) for x in range(1, 8)]))
})
try:
  crash_day_vali.validate(crash)
except pa.errors.SchemaError as err:
  print(err.failure_cases)

  index failure_case
0  None      float64


### Intra-record Check

In [1024]:
# School zone cannot be 1 for interstate highways, which functional class is 01 or 11.
school_func = crash.loc[crash['School Zone Indicator'] == '1']
school_func_vali = DataFrameSchema({
    'Functional Class Code': Column(pa.String, Check.notin(['01', '11']))
})
try:
  school_func_vali.validate(school_func)
except pa.errors.SchemaError as err:
  print(err.failure_cases)

  index failure_case
0  None      float64


### Inter-record Check

### Summary

In [1025]:
# Every crash has a unique crash ID.
assert crash['Crash ID'].nunique() == crash['Crash ID'].size

In [1026]:
# Every vehicle has a unique vehicle ID.
assert vehicle['Vehicle ID'].nunique() == vehicle['Vehicle ID'].size

### Referential Integrity

In [1027]:
# Latitude degree, minutes, and second should be exist at the same time.
latitude_vali = DataFrameSchema({
    'Latitude Degrees': Column(pa.Int, nullable=False),
    'Latitude Minutes': Column(pa.Int, nullable=False),
    'Latitude Seconds': Column(pa.Int, nullable=False),
})
try:
  latitude_vali.validate(crash)
except pa.errors.SchemaError as err:
  print(err)
  print(err.failure_cases)

expected series 'Latitude Degrees' to have type int64, got float64
  index failure_case
0  None      float64


In [1028]:
# Every participant should have a vehicle.
vehicle_exist_vali = DataFrameSchema({
    'Vehicle ID': Column(pa.String, nullable=False),
})
try:
  vehicle_exist_vali.validate(participant)
except pa.errors.SchemaError as err:
  vehicle_exist_err = err.failure_cases

### Statistical Distribution

In [1029]:
# The age of the driver is normally distributed.
participant['Age'] = participant['Age'].dropna(how='any')
participant['Age'].describe()

count    1196.000000
mean        4.667224
std         3.414411
min         0.000000
25%         2.000000
50%         2.000000
75%         9.000000
max         9.000000
Name: Age, dtype: float64

In [1030]:
# There is not much difference between the number of male and female passengers.
s = df.groupby("Sex", as_index=True)